In [28]:
import math
import numpy as np 
import matplotlib.pyplot as plt 
from numpy import random 

In [19]:
#initialize the values
N = 100
D = 1
dx = 1/N
w = 1.9
e = 10**(-5)
eta = 1

In [3]:
def update_SOR(Y, w, N = 50, objects = []):
    '''
    The SOR method grid update function
    Args:
        -Y, the grid at the current timestep
        -w, the omega constant
        -N, the size of the grid
        -objects, the range of one or two objects in the grid blocking diffusion
    Out:
        -Y, the grid at the next timestep
    '''
    #update the grid at position i, j and take boundary conditions into account
    for j in range(1, len(Y)-1):
        for i in range(len(Y)):
            if i == 0:
                 Y[j,i] = (1-w)*Y[j,i] + (w/4) * (Y[j+1,i] + Y[j-1,i] + Y[j,i+1] + Y[j,-1]) 
            elif i == N-1:
                 Y[j,i] = (1-w)*Y[j,i] + (w/4) * (Y[j+1,i] + Y[j-1,i] + Y[j,0] + Y[j,i-1]) 
            else:
                 Y[j,i] = (1-w)*Y[j,i] + (w/4) * (Y[j+1,i] + Y[j-1,i] + Y[j,i+1] + Y[j,i-1])
        if len(objects) > 0:
            Y[objects[0,0]:objects[0,1], objects[1,0]:objects[1,1]] = 0
            if len(objects) == 4:
                Y[objects[2,0]:objects[2,1], objects[3,0]:objects[3,1]] = 0    
    return(Y)  

In [4]:
def init(N):
    '''
    Initialize a grid with all zeros except on the first row where the value is all ones
    Args:
        -N the size of the grid
    Out: the initial state of the grid
    '''
    k = 1
    Y = np.zeros((N,N))
    Y[0] = np.ones(N)
    Y_prev = np.copy(Y)
    Y = update_SOR(Y, w, N)
    #calculate the difference between the last two iterations
    diff = np.abs(Y - Y_prev)
    diff_val = (np.amax(diff))
    #until the differnce is smaller than epsilon, update the grid
    while(diff_val > e):
        k = k+1
        Y_prev = np.copy(Y)
        Y = update_SOR(Y, w, N)
        diff = np.abs(Y - Y_prev)
        diff_val = (np.amax(diff))
    return Y
Y = init(N)

In [52]:
def growth(Y, seed, neighbours, eta):
    total = 0
    prop = []
    for s in seed:
        Y[s[0],s[1]] = 0
    for n in neighbours:
        total += (Y[n[0],n[1]])**eta
        c_ij = (Y[n[0],n[1]])**eta
        prop.append(c_ij)
    prop = prop/total
    
    index = random.choice(np.arange(len(neighbours)), p = prop)
    new_s = neighbours[index] 
    
    seed = np.vstack([seed, new_s])
    neighbours = np.delete(neighbours, index, axis = 0)
    for i in [-1,1]:
        if Y[new_s[0]+i,new_s[1]]!=0:
            neighbours = np.vstack([neighbours, [new_s[0]+i,new_s[1]] ])
        if Y[new_s[0],new_s[1]+i]!=0:
            neighbours = np.vstack([neighbours, [new_s[0],new_s[1]+i]] )
# fix boundary conditions 
    return Y, seed, neighbours

In [51]:
seed = np.array([[99,50]])
neighbours= np.array([[98,50], [99,49], [99,51]])
growth(Y, seed, neighbours, eta)

[[99 49]
 [99 51]]


(array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
         1.        ],
        [0.98985899, 0.98985899, 0.98985899, ..., 0.98985936, 0.98985937,
         0.98985937],
        [0.9797184 , 0.97971841, 0.97971842, ..., 0.97971916, 0.97971916,
         0.97971917],
        ...,
        [0.02015207, 0.02015207, 0.02015208, ..., 0.02015254, 0.02015254,
         0.02015255],
        [0.01007614, 0.01007615, 0.01007615, ..., 0.01007638, 0.01007638,
         0.01007638],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]]),
 array([[99, 50],
        [98, 50]]),
 array([[99, 49],
        [99, 51],
        [97, 50],
        [98, 49],
        [98, 51]]))